##Character-Based Handwritten Text Transcription with
##Attention Networks version -2

url:= https://arxiv.org/pdf/1712.04046v3

# 1. Introduction

an open problem : it is harder to segment and recognize individual charactors rather tahn words

 Moreover, transcription models must solve the problem of finding and classifying characters at
each time-step without knowing the alignment between the input sequence of image pixels
and the target sequence of characters [2].

Previous : RNN and CTC

=> the encoder-decoder network proposed by Deng et al. which extends the encoder-decoder RNNs of Vinyals et al.


The encoder-decoder
model encodes a variable-length sequence of characters into a fixed-length vector and then
decodes the vector into a variable-length target label. Encoder-decoder RNNs are suitable for
handling long sequences of data and have become standard for neural machine translation,
speech recognition [12], and image captioning [13] tasks.


Encoder-decoder RNNs are suitable for
handling long sequences of data and have become standard for neural machine translation,
speech recognition [12], and image captioning [13] tasks.


CNN : extracts visual features from images,

RNN Encoder :  re-encoders each row of the grid.

RNN Decoder : output a character sequence one step at a time, using an attention mechanism to emphasize the most important columns of re-encoded features at each decoding step.


Attention-based networks are
capable of modeling the language structures within the output sequence, rather than simply
mapping the input to the correct output [15].


The main differentiator in our approach is that we employ a CNN to extract image features and a separate RNN encoder to re-encode the features so that the encoder can learn new
features such as text directionality. Another difference is that we use an unidirectional RNN
decoder to predict the sequence of characters. Gui et al. [29] train character-aware attention
networks, but the architecture differs in that they use an attention-based bidirectional RNN
decoder and CTC output layer to convert predictions made by the decoder into a character
sequence



Character-aware
models view the input and output text lines as a sequence of characters rather than words,
and each character prediction is explicitly conditioned on the previous character. These models are capable of making inferences about unseen source words and also generating unseen

CNN : inputs x and arrange the visual features on a grid, V with dimensions H x W x C  C: channels, H,W are reduced dimensions by max pooling operations

Pool : Max pool
Bn : Batch normalization

the RNN encoder slides across each row of V, and at each time-step t, recursively update a hidden state h_t, using v_t as input:

  h_t = f(v_t, h_t-1; seata)

  f : a non linear activation
  t : a each time-step
  h_t : a hidden state

Encoding row-wise is useful for transcription tasks // the encoder can learn sequential order information // test directionally.

the encoder outputs a re-encoded feature grid hat V_h,w

Hat V := an inital hidden state, hat V_h,0

RNN decoder : a conditional languate model to give the probability of the next character given the history and re-encoded feature grid

W_1, and W_2 are learnded parameters of the model, and the softmax activatio function assigns probabilities over summation.




## Dataset, DataLoader

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import pandas as pd
from PIL import Image
import numpy as np
import os
import csv
from io import BytesIO

import pickle


"""
IAM Dataset download
"""

if not os.path.exists('/content/data/') :
    os.mkdir("/content/data")

splits = {'train': 'data/train.parquet', 'validation': 'data/validation.parquet', 'test': 'data/test.parquet'}


max_len = -1
str_len = -1
for iter in ['test', 'validation', 'train']:

    if not os.path.exists(f'/content/data/{iter}') :
        os.mkdir(f"/content/data/{iter}")
    df = pd.read_parquet("hf://datasets/Teklia/IAM-line/" + splits[iter])
    df_csv = df['text']

    with open(os.path.join("/content/data",f"{iter}.csv"), 'w', newline="") as csvfile:
        spanwriter = csv.writer(csvfile, delimiter = '\t', quotechar="|")
        spanwriter.writerow(["_path","text"])
        for id, txt in enumerate(df_csv):
            spanwriter.writerow([id,txt])
            if len(txt) > str_len:
                str_len =len(txt)

    #df_csv.to_csv(path_or_buf=os.path.join('/content/data', f'{iter}.csv'), sep='\t')

    df_imag = df['image']
    for idx, img in enumerate(df_imag):
        image = Image.open( BytesIO(img['bytes']))
        #the max width : 2083, the fixed height : 64
        image=image.resize((2084,64))
        image.save(os.path.join(f"./data/{iter}",f"{idx}.jpg" ))

        image = image.getchannel(0)
        width_size , height_size = image.size
        if(width_size > max_len):
            max_len =width_size

print(max_len)
print(str_len)

#preprocessing

We follow the image preprocessing steps of Puigcerver et al. [19, 46], which includes binarizing the images in a manner that preserves their original grayscale information [47], rescaling the images, and converting the images to JPEG format. Fig. 2 provides an example of a preprocessed image from each benchmark dataset.

46: https://github.com/jpuigcerver/Laia, gitHub repository

i findout it ...

In [ ]:
import skimage.color as img_color
from PIL import Image
import numpy as np

def preprocess(image_path):
    image = Image.open(image_path)
    image = np.array(image)
    ## img to gray
    ## Actually aready it is ...
    if len(image) ==3:
      image = img_color.rgb2gray(image)

    #normalize the image
    image = 1-image/255

    return image

In [ ]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import math

class IAMDataset():
    def __init__(self, split, nclasses =None ,root_dir = "/content/data", transforms = None, target_transforms= None):

        if  not split in ["train", "validation", 'test']:
          print("split must be among in ['train', 'validation','test']")
          return

        """
          /content/data/train/
          /content/data/train/train.csv

        """

        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, split)
        self.csv_dir = os.path.join(root_dir,f"{split}.csv")
        self.transforms = transforms
        self.target_transforms = target_transforms


        label_data = pd.read_csv(os.path.join(self.csv_dir), delimiter="\t",quotechar="\n")
        label_data.columns = ['_path','text']
        self.img_labels = label_data
        self.label_len = [len(label.strip()) for label in label_data['text'] ]
        if  nclasses:
            self.nclasses = nclasses

        else:
            char_set = set()
            cToInt = {}
            intToC = {}

            for transc in self.img_labels['text']:
                char_set.update(transc)

            char_set = list(char_set)
            char_set.sort()
            cdict = {c: (i+1) for i, c in enumerate(char_set)}
            icdict = {(i+1):c for i, c in enumerate(char_set)}

            self.nclasses = {'classes':char_set,'c2i':cdict,'i2c':icdict}


        # create dictionaries for character to index and index to character
        # 0 index is reserved for CTC blank

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, f'{self.img_labels.iloc[index,0]}.jpg')
        image = Image.open(img_path)
        img_np = np.array(image)
        image = torch.tensor(img_np)
        label = self.img_labels.iloc[index,1]

        label_len = len(label.strip())
        #label_len = label_len[:np.newaxis]

        """
          actuaclly we only use the preprocessing channel 3 to 1 rgb2gray
        """
        # 0 : padding , 1 : Go, 2: EOS, 3:UNK
        def preprocess_label(transc,tdecs):

            word=[tdecs[c]+3 for c in transc.strip()]
            word.append(2)
            word[1:] = word
            word[0] = 1

            # padd
            padd_size = int(math.floor(2084/3))
            act_len = len(word )

            word = np.concatenate((word, np.zeros( padd_size- act_len) ))
            #[694]
            return word

        def preprocess(image_path):
            image = Image.open(image_path)
            image = np.array(image)
            ## img to gray
            ## Actually aready it is ...
            if len(image) ==3:
              image = img_color.rgb2gray(image)

            #normalize the image
            image = 1-image/255

            return image


        if self.transforms:
          image = self.transforms(image)

        image = preprocess(img_path)
        image = torch.tensor(image, dtype=torch.float32).reshape(1,64,2084)
        label = preprocess_label(label,self.nclasses['c2i'])
        label =torch.tensor(label,dtype=torch.float32)


        return image, label

#NetStructure

we fix the image heignt to 64 pixels while maintain aspect ratio, group images with similar widths, and pad with whitespace to facilitate batching

CNN consists of seven convolutional layers,each followed by RELU, and them max-pooling layer,

the 3, 5, 7 layer use batch normalization following the convolution in orderto speedup training

Droupout is applied to the ouput of the seventh convolutional layer in

Table 2: CNN specification.
Conv Pool
# filters Filter size Stride size Bn Pool size Stride size
64 (3,3) (1,1) (2,2) (2,2)

128 (3,3) (1,1) (2,2) (2,2)

256 (3,3) (1,1) X - -

256 (3,3) (1,1) (2,1) (2,1)

512 (3,3) (1,1) X - -

512 (3,3) (1,1) (2,1) (2,1)

512 (2,2) (1,1) X - -

Stacked on the CNN is a single-layer, bidirectional long Short-Term Memory (BLSTM) encoder with 512 hidden units and a two-layer Gated Recurrent unit (GRU) decoder, each with 256 hidden units
The attentional decoder interprets the feature representation, focusing on the most important columns of re-encoded features


we train the networks for 200 epochs with a batch size of 8, stochastic gradient descent to learn the parameter weights, and the Adam optimizer to adapt the learning rate.


 As a regularization strategy, we implement ℓ
2
regularization loss and data augmentation by applying
random affine transformations to 20% of the training set images, including scaling, translating, rotating, and shearing

 In addition, we employ gradient norm clipping and gradient
normalization in order to prevent exploding gradients.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

"""
  Conv                                Pool
  filters Filter size Stride size Bn Pool size Stride size
    64 (3,3) (1,1)                    (2,2) (2,2)

    128 (3,3) (1,1)                   (2,2) (2,2)

    256 (3,3) (1,1)               O   (2,1) (2,1)

    256 (3,3) (1,1)                   (2,1) (2,1)

    512 (3,3) (1,1)               O   (2,1) (2,1)

    512 (3,3) (1,1)                    - -

    512 (2,2) (1,1)    "valid"    O    - -
    the size order is : (1, width)

    the adjusted network ... came from the github ... Attention-OCR

"""


class pCNN(nn.Module):

    def __init__(self , number):
        super(pCNN,self).__init__()

        if number in [3,5,7]:
            self.isBatch =True
        else :
            self.isBatch = False
        #paper say just a number of channel ... the last out
        filterNumber= [1,64,128,256,256,512,512,512]

        inputs = filterNumber[number-1]
        outputs = filterNumber[number]
        filterSize = 3
        if(number ==7):
            filterSize = 2

        pool_size = 2
        if (number >2):
            pool_size =1

        padding = 'same'
        if(number == 7):
            padding = 'valid'

        self.MaxPool =True
        if (number==6 or number ==7):
            self.MaxPool =False

        self.ConvLayer = nn.Conv2d(inputs, outputs,filterSize, padding=padding)

        self.ReLULayer = nn.ReLU()
        if(self.isBatch) :
            self.BatchLayer = nn.BatchNorm2d(outputs)

        if(self.MaxPool):
            self.nextLayer = nn.MaxPool2d((2,pool_size), stride = (2,pool_size))

    def forward(self, x):
        x= self.ConvLayer(x)
        logits= self.ReLULayer(x)
        if self.isBatch:
          logits=self.BatchLayer(logits)
        if self.MaxPool:
          logits= self.nextLayer(logits)

        return logits

class CNNnet(nn.Module):
    def __init__(self):
        super(CNNnet,self).__init__()
        self.conv1 = pCNN(1)
        self.conv2 = pCNN(2)
        self.conv3 = pCNN(3)
        self.conv4 = pCNN(4)
        self.conv5 = pCNN(5)
        self.conv6 = pCNN(6)
        self.conv7 = pCNN(7)
        self.Droplayer = nn.Dropout()

    def forward(self,x):

        #[batch, 64, 2520]
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.conv3(y)
        y = self.conv4(y)
        y = self.conv5(y)
        y = self.conv6(y)
        y = self.conv7(y)
        logits = self.Droplayer(y)
        logits=logits.squeeze()
        return logits


# Construct RNN - encoder// RNN - decoder

RNN - encoder

slides across each row of V, and at each time-step t, recursively updates a hidden state h_t, using v_t in V as input:

  h_t = f(v_t, h_t-1; seta)
  

In [ ]:
import torch
from torch import nn

class CNN_RNNEncoder(nn.Module):
    def __init__(self, num_hidden=256):
        super(CNN_RNNEncoder,self).__init__()
        """
            the part of from image process to just before of attention ...
        """

        self.CNNLayer = CNNnet()
        #[batch,channel,width]
        #[8, 256,541]
        self.LSTMLayer = nn.LSTM(512, num_hidden, 1, bidirectional=True)

    def forward(self,x):
        logits = self.CNNLayer(x)
        #[batch,channel, width]

        logits = logits.permute(2,0,1)
        #[width, batch, channel]

        width_size, batch_size, channel_size = logits.size()

        pre_encoder, (hidden_state, __)  = self.LSTMLayer(logits)
        #[width,batch,channel] , (2,batch,2*num_hidden)

        hidden_state = torch.concat([hidden_state[0],hidden_state[1]], dim=1)

        pre_encoder = pre_encoder.transpose(0,1)

        return pre_encoder, hidden_state


In [ ]:
class RNNDecoder(nn.Module):
    def __init__(self,num_hidden=256, nclasses=82):
        super(RNNDecoder,self).__init__()
        """
            attention and decoder
            we given the target_weight, target_input, decoder_input
            all word : 79 + padd, go, stop, UNK => 82

            attention_vec_size  < - attn_size  : 512
            attn_length = attention_state[1]
            attn_size = attenion_state[2] : 512
        """

        """
            First, we run the cell on a combination of the input and previous attention masks:
            cell_output, new_state = cell(linear(input, prev_attn), prev_state).
            Then, we calculate new attention masks:
            new_attn = softmax(V^T * tanh(W * attention_states + U * new_state))

            and then we calculate the output:
            output = linear(cell_output, new_attn).
            state: The state of each decoder cell the final time-step.
            It is a 2D Tensor of shape [batch_size x cell.state_size].
        """
        """
            decoder_inputs: A list of 1D batch-sized int32 Tensors (decoder inputs).
        """

        self.num_hidden = num_hidden
        # AttentionLayer
        self.attentionMat = nn.Linear(512,512)

        # it is a weigthed sum of query ...
        self.encoder_attention= nn.Conv2d(512,512,(1,1),bias=False,padding='same')
        self.attentionVec = nn.Linear(512,512, bias = False)

        self.EmbeddingToSymbol = nn.Linear(512,nclasses)
        self.EmbeddingLookup = nn.Embedding(nclasses, 512 , padding_idx=0)

        self.CombinationInputAttention = nn.Linear(512,128)

        #GRU bidirection , 2layer
        self.GRULayer_first = nn.GRUCell(128,512)
        self.GRULayer_second = nn.GRUCell(512,512)

        self.OutputProjection = nn.Linear(512,512)

    def forward(self, encoder_inputs, hidden_states):

        # the label length : 694

        # bucket[0] = 2084/4 = 541, bucket[1]= 2084/3 => 693
        # there is no bucket ... no resize, or permutate
        # attention_states = encoder_inputs, initial_state = hidden_states
        # [batch ,width,channel] ,[batch,channel]

        ## do embedding_attention_decoder !!
        # To calculate W1 * h_t we use a 1- by- 1 convolution, reen to reshape before
        # encoder_inputs < = attention_state, label_inputs <= decoder_inputs

        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

        batch_size, attn_length, attn_size = encoder_inputs.size()

        hidden = encoder_inputs

        def attention(query,hidden):
            #query = hidden_states, state
            #[batch, 512]
            batch_size, attn_size = query.size()

            hidden = hidden.permute(0,2,1)
            #[batch, attn_length, attn_size] => [batch,attn_size,attn_length]
            hidden = torch.reshape(hidden,[-1,attn_size,1,attn_length])
            #[batch, attn_length, attn_size] => [batch,attn_size, 1,attn_length ]

            y = self.attentionMat(query)
            #[batch,attn_size]

            y = torch.reshape(y,[-1,1,1,attn_size])

            #Attention mask is a softmax of v^T * tanh(...)
            hidden_features = self.encoder_attention(hidden)
            #hidden :  [batch, atten_size,1,attn_length]
            #hidden_features = [batch, atten_size,1,attn_length]

            hidden_features = hidden_features.permute(0,3,2,1)
            #[batch,attn_length,1,attn_size]

            s = torch.tanh(hidden_features +y)
            #[batch,attn_length,1,attn_size] + [batch,1,1,attn_size]
            # = > [batch,attn_length,1,attn_size]

            s = self.attentionVec(s)
            s = torch.sum(s, (2,3))                         # theta_ij
            #[batch,attn_length]

            ss = torch.softmax(s,dim=1)
            #[batch,attn_length]

            # the attention-weighted vector ds
            ds = torch.sum(torch.reshape(ss,[-1,attn_length,1,1])* hidden.transpose(1,3), [1,2])
            ds = torch.reshape(ds,[-1,attn_size])
            #[batch,attn_size]

            return ds, ss

        def loop_function(prev):
            """
                embedding and argmax ...
            """
            prev = self.EmbeddingToSymbol(prev)
            #[batch,attn_size] = > [batch, symbol]

            prev_symbol = prev.argmax(1).squeeze()

            emb_prev = self.EmbeddingLookup(prev_symbol)
            #[batch,attn_size]
            return emb_prev

        def GRULayer(input,state):
            """
              GRUCell return => new_h, new_h // cell_state and hidden_state are same
              No way to bidirection on the github
            """

            input = self.GRULayer_first(input,state)
            result = self.GRULayer_second(input,state)
            #[batch, 512]

            return result

        outputs = []
        #[694,batch, 512]
        attention_weights_history = []
        #[694,batch,512]

        state = hidden_states

        ## attn, attn_weights <- came from img  [batch, attn_size]
        ## inp <- decoder_inputs <- label [batch, 694] ?

        # the length of label decoder_inputs :  694

        prev =None
        attns = torch.zeros(batch_size,attn_size).to(device)

        for idx in range(694):
            if prev ==None:
                inp = torch.ones(batch_size,dtype=int).to(device)
                inp = self.EmbeddingLookup(inp)

            else :
                inp = loop_function(prev)
                #[batch,512]

            x = self.CombinationInputAttention(inp + attns)
            #[batch,512]
            state = GRULayer(x, state)


            attns, attn_weights = attention(state, hidden)
            attention_weights_history.append(attn_weights)

            output = torch.matmul(state+attns, self.params.get('OutputProjection'))
            output = output+ self.params.get('OutputProjectionVec')

            prev = output
            outputs.append(output)

        outputs = torch.stack(outputs, dim =0)
        #[attn_length, batch, attn_size]
        attention_weights_history = torch.stack(attention_weights_history, dim=0)
        embedd_outputs = self.EmbeddingToSymbol(outputs)
        #[attn_length, batch, symbol]

        return embedd_outputs ,state, attention_weights_history



In [ ]:
class HTRnet(nn.Module):
  def __init__(self,nclasses):
      super(HTRnet,self).__init__()
      self.EncoderLayer = CNN_RNNEncoder(256)

      self.DecoderLayer = RNNDecoder(256,nclasses=nclasses)

  def forward(self,x):

      pre_encoder_inputs,hidden_state = self.EncoderLayer(x)
      outputs, states, attention_weights_history= self.DecoderLayer(pre_encoder_inputs, hidden_state)

      return  outputs, states, attention_weights_history



#Metrix

define the loss, optimizer and score function

In [ ]:
!pip install editdistance

In [ ]:
#only we use the char dict

import editdistance

class CER :
    def __init__(self):
        self.total_dist = 0
        self.total_len =1

    def update(self, prediction, target):
        dist = float(editdistance.eval(prediction,target))
        self.total_dist += dist
        self.total_len += len(target)

    def score(self):
        return self.total_dist / self.total_len

    def reset(self):
        self.total_dist = 0
        self.total_len = 1


#Trainer, Eval


about : diffent length sequence CrossEntropy


https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [ ]:
import tqdm
from torchvision.transforms import v2

class HTRTrainer(nn.Module):
    def __init__(self):
        super(HTRTrainer, self).__init__()

        self.prepare_dataloaders()
        self.prepare_net()
        self.prepare_losses()
        self.prepare_optimizers()


    def prepare_dataloaders(self):

        transforms = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32,scale =True),
            v2.RandomAffine(degrees = (-45,45),scale = (80,120),shear = ([-16,16,-16,16])),
            v2.ToDtype(torch.uint8,scale =True),
        ])

        train_set = IAMDataset( split = "train",transforms=transforms)
        self.classes = train_set.nclasses

        # save classes in data folder
        #np.save(os.path.join("/content/data", 'classes.npy'), self.classes)

        # create dictionaries for character to index and index to character
        # 0 index is reserved for pad blank_index

        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

        val_set = IAMDataset( split = "validation",nclasses = self.classes)
        test_set = IAMDataset( split = "test", nclasses= self.classes)


        """
        num_epoch : 200
        batch_size : 8
        """

        train_loader = DataLoader(train_set, batch_size=8,shuffle=True)
        test_loader = DataLoader(test_set, batch_size=8,shuffle=True)
        val_loader = DataLoader(val_set, batch_size=8,shuffle=True)

        self.loaders ={"train": train_loader,"val": val_loader,"test": test_loader}

    def prepare_net(self):
        classes = self.classes['classes']
        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
        self.net = HTRnet(nclasses=len(classes)+4).to(device)   # 83 : 0~ 82

    def prepare_losses(self):
        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
        self.CrossLayer = nn.CrossEntropyLoss(ignore_index=0,reduction="mean").to(device)

    def prepare_optimizers(self):

        optimizer = torch.optim.Adam(self.net.parameters())
        self.optimizer = optimizer

    def decode(self, tdec, tdict,blank_id = 0, start_id = 1, end_id = 2):
        vec = []
        for c in tdec.tolist() :
            if c > 3:
                vec.append(tdict[c-3])
            elif c == end_id:
                break

        # there is a subtle thing that i didn't know ... from the decoder there is a different between Go, padd_id ?!?
        vec = ''.join(vec).strip()
        return vec

    def sample_decoding(self):

        # get a random image from the test set
        img, transcr = self.loaders['val'].dataset[np.random.randint(0, len(self.loaders['val'].dataset))]

        self.net.eval()
        with torch.no_grad():
            test_object = self.net(img)
            test_object = test_object[0]

        self.net.train()

        tdec = test_object.argmax(2).permute(1, 0).cpu().numpy().squeeze()


        dec_transcr = self.decode(tdec, self.classes['i2c'])

        print('orig:: ' + transcr.strip())
        print('pred:: ' + dec_transcr.strip())

    def train(self, epoch):

        self.net.train()
        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
        t = tqdm.tqdm(self.loaders['train'])
        t.set_description('Epoch {}'.format(epoch))

        max_grad_norm = 5.0
        for iter_idx, (img, transcr) in enumerate(t):

            self.optimizer.zero_grad()
            img = img.to(device)
            #transcr.to(device)

            outputs, states, attention_weights_history = self.net(img)
            #[attn_length,batch, symbol]    attn_length = legnth of transcr

            dec_transcr= outputs.permute(1,2,0).to('cpu')
            #transcr = transcr.transpose(0,1)
            #[batch, attn_length,sybol]
            transcr = transcr.long()
            loss = self.CrossLayer(dec_transcr, transcr)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(self.net.parameters(),max_grad_norm )
            self.optimizer.step()

    def test(self, epoch, tset='test'):

        self.net.eval()
        device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
        self.net.to(device)
        if tset=='test':
            loader = self.loaders['test']
        elif tset=='val':
            loader = self.loaders['val']
        else:
            print("not recognized set in test function")

        print('####################### Evaluating {} set at epoch {} #######################'.format(tset, epoch))

        cer = CER()
        for (imgs, transcrs) in tqdm.tqdm(loader):
            imgs = imgs.to(device)

            #transcrs = transcrs.to(device)
            with torch.no_grad():
                o, state, attention_weights_history = self.net(imgs)

            #[batch,694,80]
            tdecs = o.transpose(0,1).argmax(2).cpu().numpy().squeeze()

            for tdec, transcr in zip(tdecs, transcrs):
                transcr = self.decode(transcr, self.classes['i2c'])
                dec_transcr = self.decode(tdec, self.classes['i2c'])

                cer.update(dec_transcr, transcr)

        cer_score = cer.score()

        print('CER at epoch {}: {:.3f}'.format(epoch, cer_score))


    def save(self, epoch):
        print('####################### Saving model at epoch {} #######################'.format(epoch))
        if not os.path.exists('./saved_models'):
            os.makedirs('saved_models')

        torch.save(self.net.cpu().state_dict(), './saved_models/htrnet_{}.pt'.format(epoch))


In [ ]:
import time

htr_trainer = HTRTrainer()

max_epochs =75

print('Training Started!')

htr_trainer.test(0)

for epoch in range(1, max_epochs + 1):
    htr_trainer.train(epoch)

    # save and evaluate the current model
    if epoch % 5 == 0:
        htr_trainer.save(epoch)
        htr_trainer.test(epoch, 'test')
        htr_trainer.test(epoch, 'val')

    # save the  model
    if epoch % 25 == 0:
        if not os.path.exists('./saved_models'):
            os.makedirs('./saved_models')
        file_name= time.strftime('%Y_%m_%d', time.localtime(time .time()))
        torch.save(htr_trainer.net.cpu().state_dict(), f'./saved_models/{file_name}_train.np')

while(True):
    time.sleep(1000)
    print("hello")

In [ ]:
htr_trainer.to(device)
for epoch in range(1, max_epochs + 1):
    htr_trainer.train(epoch)

    # save and evaluate the current model
    if epoch % 5 == 0:
        htr_trainer.save(epoch)
        htr_trainer.test(epoch, 'test')
        htr_trainer.test(epoch, 'val')

    # save the  model
    if epoch % 25 == 0:
        if not os.path.exists('./saved_models'):
            os.makedirs('./saved_models')
        file_name= time.strftime('%Y_%m_%d', time.localtime(time .time()))
        torch.save(htr_trainer.net.state_dict(), f'./saved_models/{file_name}_train.np')

while(True):
    time.sleep(1000)
    print("hello")


# Encoder-decoder models and attention

url:=https://www.youtube.com/watch?v=gHk2IWivt_8&list=PLmZlBIcArwhPHmHzyM_cZJQ8_v5paQJTV
